# Pipelines + Project flow (minimal)

# Utilidades do Sklearn
- [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)



- [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)



- [Binarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Binarizer.html)



- [KBinsDiscretizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html)



- [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)



- [LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html)



- [OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)



- [SimpleImputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer)


- etc...

## Execício
Considere o seguinte dataset com alturas e pesos de algumas pessoas.

In [1]:
import pandas as pd
import numpy as np
data = pd.DataFrame(
    [
        {'gender': 'Male', 'height': 180, 'weight': 82},
        {'gender': 'Female', 'height': np.nan, 'weight': 72},
        {'gender': 'Male', 'height': 170, 'weight': 75},
        {'gender': 'Female', 'height': 165, 'weight': 60},
        {'gender': 'Male', 'height': 177, 'weight': 76},
    ])

test_data = pd.DataFrame(
    [
        {'gender': 'Male', 'height': 170, 'weight': 72},
        {'gender': 'Female', 'height': np.nan, 'weight': 60}
    ]
)

data

,gender,height,weight
0,Male,180.0,82
1,Female,NaN,72
2,Male,170.0,75
3,Female,165.0,60
4,Male,177.0,76


In [2]:
test_data

,gender,height,weight
0,Male,170.0,72
1,Female,NaN,60


**Tarefas**
1. Transformar a coluna `gender` com `OneHotEncoder`
2. Inserir dados faltantes na coluna `height`
3. Juntar as duas transformações
4. Aplicar as mesmas transformaçoes aos dados de teste
5. Criar um modelo linear que preveja o peso de uma pessoa, dados o gênero e a altura.
6. Use os dados de teste para estimar a qualidade das predições.

### Transformar a coluna gender com OneHotEncoder

In [7]:
pd.get_dummies(data['gender'])

,Female,Male
0,0,1
1,1,0
2,0,1
3,1,0
4,0,1


In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

ohe.fit(data[['gender']])

ohe.transform(data[['gender']])

<5x2 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [16]:
ohe.transform(data[['gender']]).todense()

matrix([[0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.]])

In [24]:
ohe.get_feature_names()

array(['x0_Female', 'x0_Male'], dtype=object)

### Inserir dados faltantes na coluna `height`

In [18]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')

imputer.fit(data[['height']])

imputer.transform(data[['height']])

array([[180.],
       [173.],
       [170.],
       [165.],
       [177.]])

In [19]:
data

,gender,height,weight
0,Male,180.0,82
1,Female,NaN,72
2,Male,170.0,75
3,Female,165.0,60
4,Male,177.0,76


### Juntar as duas transformações

In [21]:
data_transformed = pd.DataFrame(ohe.transform(data[['gender']]).todense(), columns=ohe.get_feature_names())
data_transformed['height'] = imputer.transform(data[['height']])
data_transformed

,x0_Female,x0_Male,height
0,0.0,1.0,180.0
1,1.0,0.0,173.0
2,0.0,1.0,170.0
3,1.0,0.0,165.0
4,0.0,1.0,177.0


In [22]:
data

,gender,height,weight
0,Male,180.0,82
1,Female,NaN,72
2,Male,170.0,75
3,Female,165.0,60
4,Male,177.0,76


### Criar um modelo linear que preveja o peso de uma pessoa, dados o gênero e a altura.


In [25]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(data_transformed, data['weight'])

LinearRegression()

### Use os dados de teste para estimar a qualidade das predições.

In [26]:
# dados teste não transformados
test_data

,gender,height,weight
0,Male,170.0,72
1,Female,NaN,60


In [27]:
model.predict(test_data[['gender', 'height']])

ValueError: could not convert string to float: 'Male'

### Aplicar as mesmas transformaçoes aos dados de teste
- Por que transformamos os dados de teste?

In [28]:
test_data

,gender,height,weight
0,Male,170.0,72
1,Female,NaN,60


In [32]:
# aplicamos só o método transform para evitar data leakage (vazamento de dados)
test_data_transformed = pd.DataFrame(ohe.transform(test_data[['gender']]).todense(), columns=ohe.get_feature_names())
test_data_transformed['height'] = imputer.transform(test_data[['height']])
test_data_transformed

,x0_Female,x0_Male,height
0,0.0,1.0,170.0
1,1.0,0.0,173.0


In [33]:
# dados de teste transformados
model.predict(test_data_transformed)

array([72.33464567, 69.76377953])

**Para pensar:** O que houve com a coluna que não foi tradada? Como adicioná-la ao resultado final?

<details>
    <summary> Depois que pensar, clique aqui! </summary>
        
> Adicionaria as colunas no dataset transformado!

</details> 

**Para pensar:** Quais as vantagens e desvantagens do método acima? Ele é escalável?

<details>
    <summary> Depois que pensar, clique aqui! </summary>
        
> Desvatangem: 
>- parece muito mais complicado (dá muito mais voltas!)
>- não é escalável: muito trabalho manual
>- por ser manual, sujeito à erros

> Vantagem:
>- fácil de aplicar nos dados de teste (só fazer .transform())
>- é "humam-readable" (perceptível, fácilmente compreensível)
    
</details>

## [Column Transformer ](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

Refaça o execício acima de forma mais 'elegante' (e segura e escalável etc)

### passos 1, 2 e 3 de uma vez só: criar X_treino

In [35]:
from sklearn.compose import ColumnTransformer

transformers_ = [('encoder', OneHotEncoder(), ['gender']), 
                 ('imputer', SimpleImputer(strategy='mean'), ['height'])]


ct = ColumnTransformer(transformers=transformers_)

# segue o padrão sklearn
ct.fit(data)


ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['gender']),
                                ('imputer', SimpleImputer(), ['height'])])

In [36]:
ct.transform(data)

array([[  0.,   1., 180.],
       [  1.,   0., 173.],
       [  0.,   1., 170.],
       [  1.,   0., 165.],
       [  0.,   1., 177.]])

### passo 4 de forma beeeeeem mais simple: criar X_teste


In [37]:
ct.transform(test_data)

array([[  0.,   1., 170.],
       [  1.,   0., 173.]])

### criar o modelo com os dados do ColumnTransformer

In [38]:
X_treino = ct.transform(data)
y_treino = data['weight']

lr = LinearRegression()

lr.fit(X_treino, y_treino) 

LinearRegression()

### Use os dados de teste para estimar a qualidade das predições.

In [42]:
from sklearn.metrics import r2_score, mean_squared_error

X_teste = ct.transform(test_data)
y_teste = test_data['weight']

y_pred = lr.predict(X_teste)


print('R2 score:', r2_score(y_teste, y_pred))
print('mean square error:', mean_squared_error(y_teste, y_pred))
## note que o resultado é exatamente igual ao resultado anterior (manual)

R2 score: -0.3256024775938433
mean square error: 47.72168919337836


**Para pensar:** Será que há um jeito elegante de fazer todo o fluxo de ML? 👇🏻

# [Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

**Tarefa:** Refaça o problema acima usando Pipeline. Note as vantagens dessa técnica!

## refazendo passos 1 a 5 com pipeline!

In [44]:
from sklearn.pipeline import Pipeline

transformers_ = [('encoder', OneHotEncoder(), ['gender']), 
                 ('imputer', SimpleImputer(strategy='mean'), ['height'])]

pre_proc_pipe = ColumnTransformer(transformers=transformers_)

model = LinearRegression()

steps_ = [('preprocessing', pre_proc_pipe), 
          ('training', model)]

pipe = Pipeline(steps=steps_)

pipe.fit(data, data['weight'])

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('encoder', OneHotEncoder(),
                                                  ['gender']),
                                                 ('imputer', SimpleImputer(),
                                                  ['height'])])),
                ('training', LinearRegression())])

## refazendo passo 6 com pipeline!
- note que os dados de teste ainda não foram preprocessados!

In [45]:
y_pred = pipe.predict(test_data)
y_teste = test_data['weight']

print('R2 score:', r2_score(y_teste, y_pred))
print('mean square error:', mean_squared_error(y_teste, y_pred))
## note que o resultado é exatamente igual ao resultado anterior (manual)

R2 score: -0.3256024775938433
mean square error: 47.72168919337836


In [55]:
from sklearn.model_selection import cross_val_score

transformers_ = [('encoder', OneHotEncoder(), ['gender']), 
                 ('imputer', SimpleImputer(strategy='constant', fill_value=162), ['height'])]

pre_proc_pipe = ColumnTransformer(transformers=transformers_)

model = LinearRegression()

steps_ = [('preprocessing', pre_proc_pipe), 
          ('training', model)]

pipe = Pipeline(steps=steps_)

cv = cross_val_score(estimator=pipe, X=data, y=data['weight'], cv=2, scoring='r2')

cv

array([-5.1861073 , -0.67203125])

**Para pensar:** Podemos aplicar mais de uma transformação em cada coluna? 👇🏻

## Pipeline + ColumnTransform
- Refaça o execício acima normalizando os dados de altura usando com MinMaxScaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler



In [ ]:
## passando a variável 'weight' sem nenhuma transformação!


# Salvando e reutilizando o modelo